<h2>Análise textual</h2>
<h5>Wellington R Monteiro</h5>

<p>Após ter modificado o dataset das reviews para que seja lido de uma forma mais efetiva pelo Redshift (e pelo Pandas), a intenção deste notebook é a de extrair informação a partir dos textos. Sendo assim, via <b>AWS Comprehend</b> todos os textos serão analisados com o objetivo de descobrir o sentimento deles e o de extrair as entidades-chave do conteúdo das mensagens. Outras análises poderiam ser realizadas utilizando o Comprehend ou outras libs como o spaCy para o treinamento de entidades customizadas ou para reconhecer a construção das frases.</p>

In [1]:
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
import spacy
import seaborn as sns
import matplotlib.pyplot as plt
from unidecode import unidecode
import re
import boto3
import json

In [2]:
dataset_folder = 'datasets/'

In [3]:
df_reviews = pd.read_csv(dataset_folder + 'olist_order_reviews_dataset_engineered.csv', delimiter=',', encoding='utf-8')
df_reviews['review_creation_date'] = pd.to_datetime(df_reviews['review_creation_date'])
df_reviews['review_answer_timestamp'] = pd.to_datetime(df_reviews['review_answer_timestamp'])

<h5>Removendo as colunas desnecessárias</h5>

In [4]:
df_reviews.drop(['review_id'], axis=1)

,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18,2018-01-18 21:46:59
1,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10,2018-03-11 03:05:13
2,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17,2018-02-18 14:36:24
3,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06
4,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01,2018-03-02 10:26:53
...,...,...,...,...,...,...
99935,22ec9f0669f784db00fa86d035cf8602,5,NaN,NaN,2017-12-09,2017-12-11 20:06:42
99936,55d4004744368f5571d1f590031933e4,5,NaN,"Excelente mochila, entrega super rápida. Super...",2018-03-22,2018-03-23 09:10:43
99937,7725825d039fc1f0ceb7635e3f7d9206,4,NaN,NaN,2018-07-01,2018-07-02 12:59:13
99938,f8bd3f2000c28c5342fedeb5e50f2e75,1,NaN,Solicitei a compra de uma capa de retrovisor c...,2017-12-15,2017-12-16 01:29:43


<h5>Ajustando espaçamentos e removendo acentos</h5>

In [5]:
df_reviews['review_comment_title'] = df_reviews['review_comment_title'].str.strip().str.lower().fillna('').apply(unidecode)
df_reviews['review_comment_message'] = df_reviews['review_comment_message'].str.strip().str.lower().fillna('').apply(unidecode)

df_reviews['title_len'] = df_reviews['review_comment_title'].str.len()
df_reviews['comment_len'] = df_reviews['review_comment_message'].str.len()

In [6]:
df_reviews.loc[df_reviews['title_len'] == 0, 'review_comment_title'] = None
df_reviews.loc[df_reviews['comment_len'] == 0, 'review_comment_message'] = None

In [7]:
df_reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,title_len,comment_len
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,None,None,2018-01-18,2018-01-18 21:46:59,0,0
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,None,None,2018-03-10,2018-03-11 03:05:13,0,0
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,None,None,2018-02-17,2018-02-18 14:36:24,0,0
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,None,recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06,0,37
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,None,parabens lojas lannister adorei comprar pela i...,2018-03-01,2018-03-02 10:26:53,0,100


<h5>Analisando via AWS Comprehend</h5>

In [8]:
available_titles = list(filter(None, df_reviews['review_comment_title'].unique().tolist()))
available_comments = list(filter(None, df_reviews['review_comment_message'].unique().tolist()))

In [9]:
comprehend = boto3.client(service_name='comprehend', region_name='us-east-2')

In [27]:
# Analisando os sentimentos do título
df_reviews['TitleSentiment'] = None
df_reviews['TitleSentimentScore'] = None

for text in available_titles:
    try:
        print(f'Processing "{text}".')
        response = comprehend.detect_sentiment(Text = text, LanguageCode = 'pt')
        df_reviews.loc[df_reviews['review_comment_title']==text, 'TitleSentiment'] = response['Sentiment']
        df_reviews.loc[df_reviews['review_comment_title']==text, 'TitleSentimentScore'] = max(response['SentimentScore'].values())
    except:
        print(f'Error processing the text "{text}".')

Processing "recomendo".
Processing "super recomendo".
Processing "nao chegou meu produto".
Processing "otimo".
Processing "muito bom.".
Processing "bom".
Processing "maravilhoso!".
Processing "entrega perfeita".
Processing "mt lindo".
Processing "otimo produto!".
Processing "recomendo sempre!!".
Processing "***** recomendo".
Processing "nao recebi o produto.".
Processing "entrega atrasou".
Processing "muito boa".
Processing "4".
Processing "medio".
Processing "prazo entrega".
Processing "super produto".
Processing "falta de produto e quebra".
Processing "nao recebi ainda".
Processing "nota3".
Processing "suspense".
Processing "entrega antes do previsto".
Processing "top".
Processing "produto show".
Processing "super recoendo".
Processing "erraram na caixa acustica".
Processing "otima compra".
Processing "horrivel".
Processing "10".
Processing "muito bom".
Processing "lamenetavel...".
Processing "qualidade ruim!".
Processing "insatisfeito".
Processing "mega recomendo".
Processing "excel

Processing "pessimo ...".
Processing "nao veio caixa".
Processing "falto intem na entrega".
Processing "supe recomendo os produto".
Processing "compra tranquila.".
Processing "minha compra venho faltan".
Processing "propaganda enganosa".
Processing "recomendo. muito bom".
Processing "colcha solteiro".
Processing "estao de parabens".
Processing "preco justo".
Processing "recomendo, com certeza".
Processing "otima empresa".
Processing "otmo vendor".
Processing "produto original".
Processing "produto nao condiz com es".
Processing "positiva".
Processing "cinco estrelas".
Processing "ressalva".
Processing "recomendo a todos!".
Processing "negativo".
Processing "9".
Processing "dentro do esperado".
Processing "8".
Processing "meu produto foi devolvido".
Processing "ficou duvida!".
Processing "bom servico".
Processing "confio muito e recomendo".
Processing "pontual".
Processing "pessimo fornecedor".
Processing "reconendo".
Processing "5 estrelas.".
Processing "agradecimento".
Processing "boa

Processing "produto quebrado!!".
Processing "mega satisfeito!!".
Processing "o produto veio bem antes".
Processing "o produto chegou bem emba".
Processing "atraso entrega".
Processing "talvez".
Processing "problema na nota fiscal".
Processing "demora na entrega".
Processing "nao fiquei satisfeito".
Processing "excelente adega!".
Processing "vendedor e produto".
Processing "satisfacao da compra".
Processing "nota 10000".
Processing "rede maravilhosa".
Processing "compra 5 estrelas.".
Processing "tablet".
Processing "produto faltando".
Processing "o arejador veio errado".
Processing "falta produtos".
Processing "nao recebi todo material".
Processing "produto de ma qualidade".
Processing "a boneca".
Processing "nao entregou".
Processing "entega rapida / bom produ".
Processing "entregue".
Processing "quero o estorno".
Processing "atrasou mas chegou".
Processing "produto".
Processing "produto devolvido".
Processing "demoro, esta aida na estr".
Processing "2 dias fora do prazo".
Processing "

Processing "nao recebimento de um pro".
Processing "otimo,eu recomendo".
Processing "relogio manchado".
Processing "excelelnte".
Processing "pedido com 10 toalhas".
Processing "nao recomendaria".
Processing "aparentemente promete".
Processing "assim como uso, recomendo".
Processing "muito legal".
Processing "entrega incompleta".
Processing "xx".
Processing "sim!".
Processing "nao recebi saia cama box".
Processing "porta temperos".
Processing "relogio euro".
Processing "pos-venda deixa a desejar".
Processing "entrega de produto.".
Processing "entrega dentro do praxp".
Processing "surpresa excelente".
Processing "prazo de entrega muito lo".
Processing "vrio com defeito".
Processing "insatisfacao parcial".
Processing "tapete".
Processing "otima !".
Processing "faltando peca".
Processing "excelente trabalho".
Processing "parabens!!!".
Processing "avaliacao de compra".
Processing "pessimo.".
Processing "rapido".
Processing "o otimo".
Processing "comprei 2 chegou 1".
Processing "falta de aco

Processing "produto nao enviado".
Processing "meu recebi o produto".
Processing "bom pelo preco".
Processing "entrega rapida!!!adoreiii".
Processing "so recebi metade do pedid".
Processing "meu produto nao foi entre".
Processing "100".
Processing "super rapido".
Processing "recomen".
Processing "o produto e de qualidade".
Processing "pelicula para camera".
Processing "encomenda".
Processing "falta entregar".
Processing "gostei e recomendo".
Processing "merecadoria errada".
Processing "muitissimo satisfeito!!!".
Processing "elegios".
Processing "problemas na entrega".
Processing "avaliando se recomendaria".
Processing "veio quebrado".
Processing "falta de um produto".
Processing "cumpre o que promete".
Processing "top!!".
Processing "produto dentro do esperad".
Processing "compromisso".
Processing "comprei dois chegou um".
Processing "muito possitivo".
Processing "torneira com filtro".
Processing "recomendaria".
Processing "auto transformador 3500".
Processing "produto nao original".
Pr

...

Processing "remessa com valor alto".
Processing "otimo produto entrega rap".
Processing "pedido 02-671663252".
Processing "nao entregou o combinado".
Processing "produto excelente!!".
Processing "falta de compromisso".
Processing "apaixonada...".
Processing "bom, porem".
Processing "lindaaaaa".
Processing "qualidade perfeita".
Processing "atendimento otimo".
Processing "1 estrela pelo produto.".
Processing "muito bommmmm".
Processing "satisfacao pelo servico".
Processing "aprovei".
Processing "ja conheco a empresa".
Processing "*************************".
Processing "100% original".
Processing "produto usado".
Processing "ofertado/pago lannister".
Processing "decepcao total".
Processing "cancelamento de compra".
Processing "recomendo totalmente".
Processing "manete".
Processing "trocar de produto".
Processing "prazo muito longo".
Processing "superrecomendado".
Processing "adesivo mapa mundi.".
Processing "ma qualidade".
Processing "compra do produto rebotec".
Processing "sem problemas"

Processing "faca".
Processing "recomendo, porem...".
Processing "reclamcao".
Processing "produto proximo de vencer".
Processing "tudo perfeito nota 10.".
Processing "super recomendo americana".
Processing "nao".
Processing "gato por lebre".
Processing "nao compre com essa loja".
Processing "defeito menos de 30 dias".
Processing "o produto e razoavel".
Processing "produto defeito".
Processing "vendedor atencioso".
Processing "em partes".
Processing "rebotec".
Processing "otimoa".
Processing "relogio casio entregue".
Processing "tesoura 5.5".
Processing "recebi veio perfeito".
Processing "revomendo".
Processing "modelo errado".
Processing "pedido faltando defeito".
Processing "adesivo otimo".
Processing "cumpre o que anuncia.".
Processing "nao vieram os parafusos".
Processing "nao cumpriu o prazo".
Processing "comprei 02, recebi 01".
Processing "super recomendo otima".
Processing "otima...".
Processing "toalheiro".
Processing "conforme anuncio".
Processing "eles sao excelentes!!".
Proces

Processing "muuito recomentadado".
Processing "avaliacao geral".
Processing "regula".
Processing "recomendo 100 por cento".
Processing "eu recomendo esta loja .".
Processing "antena".
Processing "nao posso avaliar ainda".
Processing "tudo certinho".
Processing "a nigro precisa saber".
Processing "eficaz".
Processing "propaganda nao condiz".
Processing "produto estranho".
Processing "lustre".
Processing "esperando receber".
Processing "ate o momento esta ok".
Processing "agitador consul".
Processing "relogio invicta".
Processing "perfeito.".
Processing "produto de qualidade infe".
Processing "tudo como combinado.".
Processing "xadrez".
Processing "muito rapido e bom".
Processing "cabo bom".
Processing "pedido recebido corretame".
Processing "entrega parcial do pedido".
Processing "demora no correios".
Processing "compra berco".
Processing "satisfacao completa".
Processing "muto bom!".
Processing "muito lindo !!".
Processing "otimo custo/beneficio.".
Processing "carga roubada.".
Processi

Processing "nao tive problemas".
Processing "parabens lannister".
Processing "produto pequeno".
Processing "pior experiencia. evite!".
Processing "desagradavel".
Processing "ok tudo certo recomendo !".
Processing "paguei 2 e recebi 1".
Processing "entrega boa -".
Processing "nao sei o que e pra por a".
Processing "avaliacao de entrega".
Processing "termometro portatil".
Processing "quero devolver o produto".
Processing "muito bom.....".
Processing "comprei 4 produtos e rece".
Processing "nao funciona no aparelho".
Processing "eu recomendo ta aprovado".
Processing "cuidado com a imagem".
Processing "entrega super rapida!!!!".
Processing "bandeja".
Processing "comportamento correto".
Processing "mercadoria incorreta".
Processing "produto com avarias".
Processing "recomendo 14/7/2018".
Processing "muito chateado com atraso".
Processing "recebimento parcial".
Processing "loja nota 10".
Processing "de confianca.".
Processing "super recomendavel".
Processing "otimo preco e qualidade".
Proces

Processing "preco e rapidez".
Processing "material excelente".
Processing "faltou um item".
Processing "3 falatando produto".
Processing "super recomendo!!!!".
Processing "boneca".
Processing "produto nao veio 250 ml".
Processing "fraco".
Processing "produto com imperfeicoes".
Processing "produto com medida incorr".
Processing "ganesha".
Processing "adorei o produto!".
Processing "dentro do prazo".
Processing "sabonete para mamadeira".
Processing "um lixo".
Processing "motoca".
Processing "muito bonita".
Processing "perfeito otima qualidade".
Processing "nao e tao confortavel".
Processing "tenho receios".
Processing "pedi 2 desktops chegou 1".
Processing "compra errada. arrependim".
Processing "telescopio".
Processing "vale muito a pena".
Processing "o mesmo do outback".
Processing "perfeito!!!!!".
Processing "produto nao e original".
Processing "chateado".
Processing "perfume e falso".
Processing "nao parece com a foto".
Processing "indico esse produto".
Processing "boa compra. recome

In [31]:
df_reviews.to_csv('bak.csv', index=False)

In [18]:
# Analisando os sentimentos e entidades-chave do comentário
df_reviews['ContentSentiment'] = None
df_reviews['ContentSentimentScore'] = None
df_reviews['ContentKeyEntities'] = None

count_comments = len(available_comments)
for i in range(0, count_comments, 25):
    # mostrando o índice ao invés do texto uma vez que o volume da análise é maior em relação aos títulos
    # usando o processamento em lote ao invés de processar individualmente
    
    max_i = min(count_comments, i + 25)
    print(f'Processing {i} to {max_i} of {count_comments}.')
    
    text_list = available_comments[i:max_i]
    try:
        response = comprehend.batch_detect_sentiment(TextList = text_list, LanguageCode = 'pt')
        for j in response['ResultList']:
            text = available_comments[i+j['Index']]
            df_reviews.loc[df_reviews['review_comment_message']==text, 'ContentSentiment'] = j['Sentiment']
            df_reviews.loc[df_reviews['review_comment_message']==text, 'ContentSentimentScore'] = max(j['SentimentScore'].values())
    except:
        print(f'Error processing the sentiment for the comment numbers {i} to {max_i}.')
    
    try:
        response = comprehend.batch_detect_key_phrases(TextList = text_list, LanguageCode = 'pt')
        for j in response['ResultList']:
            text = available_comments[i+j['Index']]
            df_reviews.loc[df_reviews['review_comment_message']==text, 'ContentKeyEntities'] = ','.join(list(set([x['Text'].replace(',', '') for x in j['KeyPhrases']])))
    except:
        print(f'Error processing the key phrases for the comment numbers {i} to {max_i}.')

Processing 0 to 25 of 35588.
Processing 25 to 50 of 35588.
Processing 50 to 75 of 35588.
Processing 75 to 100 of 35588.
Processing 100 to 125 of 35588.
Processing 125 to 150 of 35588.
Processing 150 to 175 of 35588.
Processing 175 to 200 of 35588.
Processing 200 to 225 of 35588.
Processing 225 to 250 of 35588.
Processing 250 to 275 of 35588.
Processing 275 to 300 of 35588.
Processing 300 to 325 of 35588.
Processing 325 to 350 of 35588.
Processing 350 to 375 of 35588.
Processing 375 to 400 of 35588.
Processing 400 to 425 of 35588.
Processing 425 to 450 of 35588.
Processing 450 to 475 of 35588.
Processing 475 to 500 of 35588.
Processing 500 to 525 of 35588.
Processing 525 to 550 of 35588.
Processing 550 to 575 of 35588.
Processing 575 to 600 of 35588.
Processing 600 to 625 of 35588.
Processing 625 to 650 of 35588.
Processing 650 to 675 of 35588.
Processing 675 to 700 of 35588.
Processing 700 to 725 of 35588.
Processing 725 to 750 of 35588.
Processing 750 to 775 of 35588.
Processing 775 t

Processing 6100 to 6125 of 35588.
Processing 6125 to 6150 of 35588.
Processing 6150 to 6175 of 35588.
Processing 6175 to 6200 of 35588.
Processing 6200 to 6225 of 35588.
Processing 6225 to 6250 of 35588.
Processing 6250 to 6275 of 35588.
Processing 6275 to 6300 of 35588.
Processing 6300 to 6325 of 35588.
Processing 6325 to 6350 of 35588.
Processing 6350 to 6375 of 35588.
Processing 6375 to 6400 of 35588.
Processing 6400 to 6425 of 35588.
Processing 6425 to 6450 of 35588.
Processing 6450 to 6475 of 35588.
Processing 6475 to 6500 of 35588.
Processing 6500 to 6525 of 35588.
Processing 6525 to 6550 of 35588.
Processing 6550 to 6575 of 35588.
Processing 6575 to 6600 of 35588.
Processing 6600 to 6625 of 35588.
Processing 6625 to 6650 of 35588.
Processing 6650 to 6675 of 35588.
Processing 6675 to 6700 of 35588.
Processing 6700 to 6725 of 35588.
Processing 6725 to 6750 of 35588.
Processing 6750 to 6775 of 35588.
Processing 6775 to 6800 of 35588.
Processing 6800 to 6825 of 35588.
Processing 682

Processing 12025 to 12050 of 35588.
Processing 12050 to 12075 of 35588.
Processing 12075 to 12100 of 35588.
Processing 12100 to 12125 of 35588.
Processing 12125 to 12150 of 35588.
Processing 12150 to 12175 of 35588.
Processing 12175 to 12200 of 35588.
Processing 12200 to 12225 of 35588.
Processing 12225 to 12250 of 35588.
Processing 12250 to 12275 of 35588.
Processing 12275 to 12300 of 35588.
Processing 12300 to 12325 of 35588.
Processing 12325 to 12350 of 35588.
Processing 12350 to 12375 of 35588.
Processing 12375 to 12400 of 35588.
Processing 12400 to 12425 of 35588.
Processing 12425 to 12450 of 35588.
Processing 12450 to 12475 of 35588.
Processing 12475 to 12500 of 35588.
Processing 12500 to 12525 of 35588.
Processing 12525 to 12550 of 35588.
Processing 12550 to 12575 of 35588.
Processing 12575 to 12600 of 35588.
Processing 12600 to 12625 of 35588.
Processing 12625 to 12650 of 35588.
Processing 12650 to 12675 of 35588.
Processing 12675 to 12700 of 35588.
Processing 12700 to 12725 of

Processing 17725 to 17750 of 35588.
Processing 17750 to 17775 of 35588.
Processing 17775 to 17800 of 35588.
Processing 17800 to 17825 of 35588.
Processing 17825 to 17850 of 35588.
Processing 17850 to 17875 of 35588.
Processing 17875 to 17900 of 35588.
Processing 17900 to 17925 of 35588.
Processing 17925 to 17950 of 35588.
Processing 17950 to 17975 of 35588.
Processing 17975 to 18000 of 35588.
Processing 18000 to 18025 of 35588.
Processing 18025 to 18050 of 35588.
Processing 18050 to 18075 of 35588.
Processing 18075 to 18100 of 35588.
Processing 18100 to 18125 of 35588.
Processing 18125 to 18150 of 35588.
Processing 18150 to 18175 of 35588.
Processing 18175 to 18200 of 35588.
Processing 18200 to 18225 of 35588.
Processing 18225 to 18250 of 35588.
Processing 18250 to 18275 of 35588.
Processing 18275 to 18300 of 35588.
Processing 18300 to 18325 of 35588.
Processing 18325 to 18350 of 35588.
Processing 18350 to 18375 of 35588.
Processing 18375 to 18400 of 35588.
Processing 18400 to 18425 of

Processing 23425 to 23450 of 35588.
Processing 23450 to 23475 of 35588.
Processing 23475 to 23500 of 35588.
Processing 23500 to 23525 of 35588.
Processing 23525 to 23550 of 35588.
Processing 23550 to 23575 of 35588.
Processing 23575 to 23600 of 35588.
Processing 23600 to 23625 of 35588.
Processing 23625 to 23650 of 35588.
Processing 23650 to 23675 of 35588.
Processing 23675 to 23700 of 35588.
Processing 23700 to 23725 of 35588.
Processing 23725 to 23750 of 35588.
Processing 23750 to 23775 of 35588.
Processing 23775 to 23800 of 35588.
Processing 23800 to 23825 of 35588.
Processing 23825 to 23850 of 35588.
Processing 23850 to 23875 of 35588.
Processing 23875 to 23900 of 35588.
Processing 23900 to 23925 of 35588.
Processing 23925 to 23950 of 35588.
Processing 23950 to 23975 of 35588.
Processing 23975 to 24000 of 35588.
Processing 24000 to 24025 of 35588.
Processing 24025 to 24050 of 35588.
Processing 24050 to 24075 of 35588.
Processing 24075 to 24100 of 35588.
Processing 24100 to 24125 of

Processing 29125 to 29150 of 35588.
Processing 29150 to 29175 of 35588.
Processing 29175 to 29200 of 35588.
Processing 29200 to 29225 of 35588.
Processing 29225 to 29250 of 35588.
Processing 29250 to 29275 of 35588.
Processing 29275 to 29300 of 35588.
Processing 29300 to 29325 of 35588.
Processing 29325 to 29350 of 35588.
Processing 29350 to 29375 of 35588.
Processing 29375 to 29400 of 35588.
Processing 29400 to 29425 of 35588.
Processing 29425 to 29450 of 35588.
Processing 29450 to 29475 of 35588.
Processing 29475 to 29500 of 35588.
Processing 29500 to 29525 of 35588.
Processing 29525 to 29550 of 35588.
Processing 29550 to 29575 of 35588.
Processing 29575 to 29600 of 35588.
Processing 29600 to 29625 of 35588.
Processing 29625 to 29650 of 35588.
Processing 29650 to 29675 of 35588.
Processing 29675 to 29700 of 35588.
Processing 29700 to 29725 of 35588.
Processing 29725 to 29750 of 35588.
Processing 29750 to 29775 of 35588.
Processing 29775 to 29800 of 35588.
Processing 29800 to 29825 of

Processing 34825 to 34850 of 35588.
Processing 34850 to 34875 of 35588.
Processing 34875 to 34900 of 35588.
Processing 34900 to 34925 of 35588.
Processing 34925 to 34950 of 35588.
Processing 34950 to 34975 of 35588.
Processing 34975 to 35000 of 35588.
Processing 35000 to 35025 of 35588.
Processing 35025 to 35050 of 35588.
Processing 35050 to 35075 of 35588.
Processing 35075 to 35100 of 35588.
Processing 35100 to 35125 of 35588.
Processing 35125 to 35150 of 35588.
Processing 35150 to 35175 of 35588.
Processing 35175 to 35200 of 35588.
Processing 35200 to 35225 of 35588.
Processing 35225 to 35250 of 35588.
Processing 35250 to 35275 of 35588.
Processing 35275 to 35300 of 35588.
Processing 35300 to 35325 of 35588.
Processing 35325 to 35350 of 35588.
Processing 35350 to 35375 of 35588.
Processing 35375 to 35400 of 35588.
Processing 35400 to 35425 of 35588.
Processing 35425 to 35450 of 35588.
Processing 35450 to 35475 of 35588.
Processing 35475 to 35500 of 35588.
Processing 35500 to 35525 of

In [19]:
df_reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,title_len,comment_len,TitleSentiment,TitleSentimentScore,ContentSentiment,ContentSentimentScore,ContentKeyEntities
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18,2018-01-18 21:46:59,0,0,NaN,NaN,None,None,None
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10,2018-03-11 03:05:13,0,0,NaN,NaN,None,None,None
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17,2018-02-18 14:36:24,0,0,NaN,NaN,None,None,None
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06,0,37,NaN,NaN,POSITIVE,0.834179,prazo
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,parabens lojas lannister adorei comprar pela i...,2018-03-01,2018-03-02 10:26:53,0,100,NaN,NaN,POSITIVE,0.951269,"internet,todos,e pratico,parabens,feliz pascoa..."


<h5>Salvando os resultados em um novo arquivo</h5>

In [23]:
df_reviews.to_csv(dataset_folder + 'reviews_comprehend.csv', index=False, sep=',', encoding='utf-8')